# This needs reworking, but it's a start

In [1]:
import pandas as pd
import sklearn
import numpy

import spacy
nlp = spacy.load('en_core_web_lg')
import skillNer
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from skillNer.skill_extractor_class import SkillExtractor

import transformers

/Users/danmarino/opt/anaconda3/envs/job_desc_project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Using SkillNer to extract skills
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

def extract_skills(text):
    annotations = skill_extractor.annotate(text)
    return annotations

def clean_extraction(annotations):
    # Initialize an empty list to store the skills
    cleaned_skills = []
    
    # Extract full matches
    for match in annotations['results']['full_matches']:
        try:
            skill = match['doc_node_value']
            score = match['score']
        except:
            skill = 'No skills detected'
            score = 0
        cleaned_skills.append((skill, score))
    
    # Extract ngram scored matches
    for match in annotations['results']['ngram_scored']:
        try:
            skill = match['doc_node_value']
            score = match['score']
        except:
            skill = 'No skills detected'
            score = 0
        cleaned_skills.append((skill, score))
    
    return cleaned_skills


# Combine the two functions with exception handling
def full_scan(text):
    try:
        return(clean_extraction(extract_skills(text)))
    except:
        return([('No skills detected', 0)])


loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [3]:
apps = pd.read_parquet('../../Data/split_1.parquet')

In [4]:
apps.columns

Index(['OpportunityId', 'ApplicationId', 'ExternalBriefDescription',
       'ExternalDescription', 'Title', 'JobCategoryName', 'IsRejected',
       'IsCandidateInternal', 'BehaviorCriteria', 'MotivationCriteria',
       'EducationCriteria', 'LicenseAndCertificationCriteria', 'SkillCriteria',
       'WorkExperiences', 'Educations', 'LicenseAndCertifications', 'Skills',
       'Motivations', 'Behaviors', 'StepId', 'StepName', 'Tag', 'StepGroup',
       'pass_first_step'],
      dtype='object')

In [5]:
apps.sample(2)

,OpportunityId,ApplicationId,ExternalBriefDescription,ExternalDescription,Title,JobCategoryName,IsRejected,IsCandidateInternal,BehaviorCriteria,MotivationCriteria,...,Educations,LicenseAndCertifications,Skills,Motivations,Behaviors,StepId,StepName,Tag,StepGroup,pass_first_step
11357,4J/MIGPV8UKn1+HEExZ/ZA==,XukP1tJb10+DZ0raI/W0Cg==,As one of our Leasing Consultants your day is ...,<p><strong><em>....Tag you're it!!!!!....</em>...,Leasing Consultant,Sales,False,False,[{'Description': 'Shows intense and eager enjo...,[{'Description': 'Inspired to perform well by ...,...,"[{'Degree': 'High School Diploma', 'Descriptio...",[],"[{'ScaleValue': '5', 'ScaleValueName': 'Expert...",[],[],BGEezloW4EGuGdzEobmLsw==,Being Reviewed,NaN,other,True
11517,a5fphd2eukG0wfws8S9zOg==,zG2xH5Jl9kWutq36WUZ65w==,Clark Pest Control is currently seeking a Pest...,"<p>Since 1950, Clark Pest Control has made the...",Service Technician,Outside Service Technician,False,False,[{'Description': 'Shows firm and constant supp...,[{'Description': 'Inspired to perform well by ...,...,"[{'Degree': 'High School Diploma', 'Descriptio...",[],"[{'ScaleValue': '2', 'ScaleValueName': 'Some K...",[{'Description': 'Inspired to perform well by ...,[{'Description': 'Devoted to a task or purpose...,7gtb5q8Nx02kb5PNR+9EJA==,Hiring Manager Review,3.0,other,True


In [11]:
apps['Full_Description_Skills'] = apps['ExternalDescription'].apply(lambda x: full_scan(x))
apps['Brief_Description_Skills'] = apps['ExternalBriefDescription'].apply(lambda x: full_scan(x))

In [12]:
apps[['Full_Description_Skills', 'Brief_Description_Skills']]

,Full_Description_Skills,Brief_Description_Skills
0,"[(No skills detected, 0)]","[(No skills detected, 0)]"
1,"[(No skills detected, 0)]","[(No skills detected, 0)]"
2,"[(No skills detected, 0)]","[(No skills detected, 0)]"
3,"[(No skills detected, 0)]","[(No skills detected, 0)]"
4,"[(No skills detected, 0)]","[(No skills detected, 0)]"
...,...,...
22048,"[(No skills detected, 0)]","[(No skills detected, 0)]"
22049,"[(No skills detected, 0)]","[(No skills detected, 0)]"
22050,"[(No skills detected, 0)]","[(No skills detected, 0)]"
22051,"[(No skills detected, 0)]","[(No skills detected, 0)]"


In [ ]:
# columns_to_combine = []
# def combine_skills(row):
#     combined = []
    
#     # List of columns to combine
#     columns = [columns_to_combine]
    
#     # Iterate over each column
#     for col in columns:
#         # Add the skills to the combined list
#         combined.extend([pair[0] for pair in row[col]])
    

#     return combined

# apps['applicant_combined_skills'] = apps.apply(combine_skills, axis=1)

In [8]:
def extract_skills(row):
    # Extract the first element (i.e., the skill) from each pair in the list
    skills_full_description = [pair[0] for pair in row['ExternalDescription']]
    #skills_brief_description = [pair[0] for pair in row['ExternalBriefDescription']]
    return skills_full_description#, skills_brief_description
apps['required_skills_full_desc'] = apps.apply(extract_skills, axis=1) #, apps['required_skills_brief_desc']

In [9]:
apps

,OpportunityId,ApplicationId,ExternalBriefDescription,ExternalDescription,Title,JobCategoryName,IsRejected,IsCandidateInternal,BehaviorCriteria,MotivationCriteria,...,LicenseAndCertifications,Skills,Motivations,Behaviors,StepId,StepName,Tag,StepGroup,pass_first_step,required_skills_full_desc
0,MbzeABKVn06G8irkoHJeIg==,nTzdqGj020CYqTouPocGSg==,"$16.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$16.00 Per Hour</strong></p>\n<p><s...,Customer Service Specialist,Customer Service,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,[],"[{'ScaleValue': '4', 'ScaleValueName': 'Advanc...",[{'Description': 'Inspired to perform well by ...,[{'Description': 'Devoted to a task or purpose...,K8yQlic+/UiXxBMpOnAoLQ==,Decline,2.0,declined,False,"[<, p, >, <, s, t, r, o, n, g, >, $, 1, 6, ., ..."
1,7SPt0A57/kyzM9hE9vxDRg==,QVk5MFCZ70WvlZE9FzAW9g==,"$15.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$15.00 Per Hour</strong></p>\n<p><s...,Customer Service Specialist,Customer Service,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,[],"[{'ScaleValue': '5', 'ScaleValueName': 'Expert...",[],[],K8yQlic+/UiXxBMpOnAoLQ==,Decline,2.0,declined,False,"[<, p, >, <, s, t, r, o, n, g, >, $, 1, 5, ., ..."
2,7SPt0A57/kyzM9hE9vxDRg==,I1kcPlAw3E+rqceh0qrutQ==,"$15.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$15.00 Per Hour</strong></p>\n<p><s...,Customer Service Specialist,Customer Service,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,[],"[{'ScaleValue': '4', 'ScaleValueName': 'Advanc...",[],[],K8yQlic+/UiXxBMpOnAoLQ==,Decline,2.0,declined,False,"[<, p, >, <, s, t, r, o, n, g, >, $, 1, 5, ., ..."
3,zolSWBFjWESbfkj8AXLYwA==,VTCXZK6/ZUWJDpxTcm2CRg==,"$15.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$15.00 Per Hour</strong></p>\n<p><s...,Customer Service Specialist,Customer Service,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,[],"[{'ScaleValue': '5', 'ScaleValueName': 'Expert...",[],[],K8yQlic+/UiXxBMpOnAoLQ==,Decline,2.0,declined,False,"[<, p, >, <, s, t, r, o, n, g, >, $, 1, 5, ., ..."
4,zolSWBFjWESbfkj8AXLYwA==,I6KgcL0jdkG8wBnL1+BZ/g==,"$15.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$15.00 Per Hour</strong></p>\n<p><s...,Customer Service Specialist,Customer Service,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,[],"[{'ScaleValue': '5', 'ScaleValueName': 'Expert...",[],[],K8yQlic+/UiXxBMpOnAoLQ==,Decline,2.0,declined,False,"[<, p, >, <, s, t, r, o, n, g, >, $, 1, 5, ., ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22048,fxoZVEqPEE2kCzzB5dR0gg==,kKyn5HF630SxbcUgs4NjEQ==,The CSR has the primary responsibility of hand...,The CSR has the primary responsibility of hand...,CUSTOMER SERVICE ASSOCIATE-MEDCARE EQUIPMENT C...,SALES / CUSTOMER SERVICE,False,False,[{'Description': 'Works well as a member of a ...,[],...,[],"[{'ScaleValue': '5', 'ScaleValueName': 'Expert...",[],[],ybIHHaXmj0uuLZgq0cuTcw==,Being Reviewed,NaN,other,True,"[T, h, e, , C, S, R, , h, a, s, , t, h, e, ..."
22049,zxzAYt+mdUisEi0bpPoIwg==,Db9QoDRyhEuuLyeLlAQLNg==,The CSR has the primary responsibility of hand...,The CSR has the primary responsibility of hand...,CUSTOMER SERVICE ASSOCIATE-MEDCARE EQUIPMENT C...,SALES / CUSTOMER SERVICE,False,False,[{'Description': 'Works well as a member of a ...,[],...,[],[],[],[],+a564i63Yk28Y6Po9vewlw==,Assessment Requested,NaN,other,True,"[T, h, e, , C, S, R, , h, a, s, , t, h, e, ..."
22050,DaF4IZR2vkybg4Ar7SLVBg==,dYghRCNqc0SUzUK6eyH2Ug==,"Effectively interacts with physician offices, ...",<p>Effectively interacts with physician office...,SALES SUPPORT SPECIALIST - MedCare - Greensbur...,SALES / CUSTOMER SERVICE,False,Fals

In [ ]:
def calculate_intersection(row):
    # Use Python's built-in set data structure to calculate the intersection
    intersection = list(set(row['required_skills']) & set(row['applicant_combined_skills']))
    return intersection
apps['intersection'] = apps.apply(calculate_intersection, axis=1)

In [ ]:
apps['intersection_length'] = apps['intersection'].apply(lambda x: len(x))
apps[['intersection_length']].describe()

In [ ]:
apps[['required_skills', 'applicant_combined_skills', 'intersection']]

In [ ]:
apps.to_pickle('extracted_skills.pkl')

# What we have now -
- Cleaned up data with the results of their applications (We know when people have gotten past the resume stage (<b>make a quick program to extract that - an EDA - May take a day or so</b>))
- We have skills extracted (hopefully?)
- We have cosine simialrity using BERT and the corresponding semantic ranking (thanks to cosine similarity)

# What we should do
- Define the parameters. What does our model take in, and what does the JSON it returns look like? (few hours)
- Make a big ol program with our functions to get us where we need to go.
- Streamlit app using the functions. Small but MVP (1d)

# What we could do
- Fine-tune BERT on our data
- The Job Title Normalization is a project on its own. It is currently unwise to pursue.
- Does the opportunity ask for a degree? Does the applicant have that degree.

Meeting with Sima - What I remember to do

1. Fix the errors
- delete the data
- Get rid of empty cells
- 

2. Find or recreate the skill-extraction notebook
- Just needs the rework

3. Define our model - Write this down (inputs, outputs)

4. Create a file full of the functions for this repo

5. Streamlit Apps!